## Colab Mount Drive


In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


## Set Main Pathes

In [1]:
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False



In [2]:
import platform

if platform.system() == "Linux":
    if is_colab():
      print("Running in Google Colab")
      path_main_folder = "/content/drive/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/TrafficManager/"


else:
    print("Running on Windows")
    path_main_folder = "I:/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/TrafficManager/"


path_data_folder = path_main_folder + "AIST_Cleaned/data2_mosheerIsmail/"#"AIST_Cleaned/data2_mosheerIsmail/"
path_cfg = path_data_folder +"cfg.sumocfg"
path_project_folder = path_main_folder + ""



Running on Windows


In [3]:
# prompt: check if file path ok

import os

if os.path.exists(path_main_folder):
  print(f"File exists at: {path_main_folder}")
else:
  print(f"File does not exist at: {path_main_folder}")


File exists at: I:/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/TrafficManager/


In [4]:
yaml_file = path_project_folder + "config.yaml"
keys_file = path_project_folder + "keys.env"

##  install sumo

In [ ]:
!sudo add-apt-repository -y ppa:sumo/stable
!sudo apt-get update
!sudo apt-get install -y sumo sumo-tools sumo-doc


In [ ]:
!source ~/.bashrc


In [ ]:
!sumo --version


## Install Packages

In [ ]:
!pip install traci
!pip install dotenv
!pip install ray
!pip install ray[rllib]
!pip install colorama
!pip install optuna
!pip install stable_baselines3

## Import Packages

In [5]:
import sys
sys.path.append(path_project_folder)


In [6]:
from Connections import SumoConnection
from Connections.Connection import *
import os
import traci
import gymnasium as gym
from numpy import inf
import numpy as np
import SumoEnvSingleAgent
from Utils_reporting import *
from Utils_running_singleAgent import *
from rewards import *
from stable_baselines3 import PPO ,DQN
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import torch
import time
from Callbacks import *
from models.d3qn import D3QNAgent
from Utils_reporting import *


## Set Parameters

In [7]:
max_step = 4000
precent_Scale = .14
is_gui = True

In [8]:
scale = int(10*precent_Scale)

## Open Sumo

In [9]:
if platform.system() == "Linux":
    if is_colab():
        print("Running in Google Colab")
        sumo_home = str(os.getenv("sumo_home_lin"))
        sumo_binary = str(os.getenv("sumo_binary_lin"))
        if is_gui:
            print("GUI not supported on linux")
    else:
      print("Not running in HPC Linux (default linux)")


else:
    print("Running on Windows")
    print("Running in Google Colab")
    sumo_home = str(os.getenv("sumo_home_win"))
    if is_gui:
        sumo_binary = str(os.getenv("sumo_binary_gui_win"))
    else:
        sumo_binary = str(os.getenv("sumo_binary_cli_win"))




Running on Windows
Running in Google Colab


In [10]:
python_path =sumo_home
data_path = path_data_folder

In [11]:
cmd=[sumo_binary, "-c", path_cfg ,"--log", "sumo_log.txt", "--verbose", "true"]


In [12]:
conn=SumoConnection.SumoConnection(cmd)

No connection found, creating a new connection ❌


## Load Traffic Lights info

In [13]:
agent_id='1698478721'

In [14]:
traffic_lights,policies=get_traffic_lights_policies_high_group(durations=[1],agent_ids=[agent_id])
traffic_lights

[(4, '1698478721')]

## Start Simulation

In [15]:
traci.simulation.setScale(scale)

In [28]:
traci.trafficlight.getRedYellowGreenState(traffic_lights[0][1])

'GGrr'

In [29]:
traci.trafficlight.getIDList()

('1698478721', '295959999', '5375841071')

In [16]:
time_before =time.time()

for i in range(500):

    traci.trafficlight.setRedYellowGreenState( '295959999','g')
    traci.trafficlight.setRedYellowGreenState( '5375841071','gggg')
    traci.trafficlight.setRedYellowGreenState( traffic_lights[0][1],'gggg')

    traci.simulation.step()
    #conn.do_steps_duration(1,max_step,traffic_lights[0][1])

time_after =time.time()


In [17]:
traci.trafficlight.getControlledLanes('5375841071')# traffic_lights[0][1])

('765377840#0_0', '765377840#0_0', '599453178#2_0', '599453178#2_0')

In [48]:
traci.vehicle.getLaneID('341')

'178936306#0_0'

In [ ]:
'178936306#1_0'=='178936306#0_0'

False

In [18]:
getSumoSensors_full( '5375841071')#traffic_lights[0][1])

(0.0, 0.0, 100.0, 0.0, 2.5, 2.5, 0.8783636666881245)

In [23]:
traci.vehicle.getSpeed('312')

0.0

In [19]:
traci.lane.getLastStepVehicleIDs( '765377840#0_0')

('144', '366', '142', '106', '103')

In [22]:

def getSumoSensors_full(tl_id):
    """
    Retrieves various traffic-related metrics from SUMO for a given traffic light ID.
    
    This function collects data such as vehicle speeds, waiting times, queue lengths, 
    throughput, and occupancy for all lanes controlled by the specified traffic light.

    Parameters:
        tl_id (str): The ID of the traffic light in SUMO.

    Returns:
        tuple: A tuple containing the following metrics:
            - avg_speed (float): Average speed of vehicles.
            - var_speed (float): Variance in vehicle speeds.
            - avg_waiting_time (float): Average waiting time of vehicles.
            - var_waiting_time (float): Variance in waiting time.
            - avg_throughput (float): Average number of vehicles passing per step.
            - avg_queue_length (float): Average number of halted vehicles.
            - avg_Occupancy (float): Average lane occupancy percentage.
    """

    vehicle_waiting = []  # Stores waiting times of vehicles
    vehicle_speeds = []  # Stores speeds of vehicles
    edges = []  # Stores unique edge IDs corresponding to lanes
    throughputs = []  # Stores number of vehicles passing per step
    queue_lengths = []  # Stores number of halted vehicles per edge
    Occupancies = []  # Stores lane occupancy percentages
    
    # Initialize output variables
    avg_waiting_time = 0
    var_waiting_time = 0
    avg_speed = 0
    var_speed = 0
    avg_throughput = 0
    avg_queue_length = 0
    avg_Occupancy = 0  # Percentage of occupied space in lanes

    # Get all lanes controlled by the traffic light
    controlled_lanes = traci.trafficlight.getControlledLanes(tl_id)

    for lane in controlled_lanes:
        # Get vehicle IDs currently in the lane
        vehicle_ids = traci.lane.getLastStepVehicleIDs(lane)

        # Collect speed and waiting time of vehicles in the lane
        vehicle_speeds += [traci.vehicle.getSpeed(vehicle_id) for vehicle_id in vehicle_ids]
        vehicle_waiting += [traci.vehicle.getAccumulatedWaitingTime(vehicle_id) for vehicle_id in vehicle_ids]

        # Identify the corresponding edge for this lane
        corresponding_edge = traci.lane.getEdgeID(lane)

        # Avoid duplicate data collection for the same edge
        if corresponding_edge not in edges:
            edges.append(corresponding_edge)
            throughputs.append(traci.edge.getLastStepVehicleNumber(corresponding_edge))  # Number of vehicles passing
            queue_lengths.append(traci.edge.getLastStepHaltingNumber(corresponding_edge))  # Number of halted vehicles
            Occupancies.append(traci.edge.getLastStepOccupancy(corresponding_edge))  # Lane occupancy percentage

    # Convert lists to NumPy arrays for efficient computation
    vehicle_speeds = np.array(vehicle_speeds)
    avg_speed = np.mean(vehicle_speeds) if vehicle_speeds.size > 0 else 0
    var_speed = np.var(vehicle_speeds) if vehicle_speeds.size > 0 else 0
    
    vehicle_waiting = np.array(vehicle_waiting)
    avg_waiting_time = np.mean(vehicle_waiting) if vehicle_waiting.size > 0 else 0
    var_waiting_time = np.var(vehicle_waiting) if vehicle_waiting.size > 0 else 0
        
    avg_throughput = np.mean(throughputs) if throughputs else 0
    avg_queue_length = np.mean(queue_lengths) if queue_lengths else 0
    avg_Occupancy = np.mean(Occupancies) if Occupancies else 0

    #clean memory
    del(vehicle_waiting)
    del(vehicle_speeds)
    del(edges)
    del(throughputs)
    del(queue_lengths)
    del(Occupancies)
    
    return (avg_speed, var_speed, avg_waiting_time, var_waiting_time, avg_throughput, avg_queue_length, avg_Occupancy)



## Results

In [21]:
time_diff = time_after - time_before
print(Fore.GREEN + f"Time taken for training: {round(time_diff,3)} seconds ({round(time_diff/60,3)} Minutes)" + Style.RESET_ALL)


Time taken for training: 0.431 seconds (0.007 Minutes)


In [22]:
def append_to_file(filename, text):
    with open(filename, 'a') as file:
        file.write(text + '\n')  # '\n' لتضيف السطر الجديد بعد كل جملة


In [23]:
traci.close()

In [24]:
results = get_sumo_statics(python_path, data_path)
for key, value in results.items():
    print(Fore.CYAN + f"{key}: {round(value,3)}" + Style.RESET_ALL)
    #append_to_file("output1.txt",f"{key}: {round(value,3)}")


Successfully ran C:\Program Files (x86)\Eclipse\Sumo\tools\xml\xml2csv.py with I:\My Drive\study\graduation_project\final\Code\project_files\TrafficManager\TrafficManager\AIST_Cleaned\data2\osm.statistics.xml
Waiting_Time: 293.26
Speed: 7.38
Waiting_Vehicles: 1066.0
TimeLoss: 308.27
Depart_Delay: 304.62
